In [1]:
# Initial imports.
import pandas as pd
from path import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score,classification_report
from imblearn.metrics import classification_report_imbalanced
import numpy as np
import pymysql

In [2]:
#conn=pymysql.connect(host='localhost',port=int(3306),user='root',passwd='101Monkeybob!',db='Final_Project')
#df=pd.read_sql_query("SELECT * FROM 'house' ",conn)


In [3]:
# Import Clean House Data
house_data = pd.read_csv("Raw Data/clean_house_data.csv",low_memory = False)
PRIME_data = pd.read_csv('Raw Data/PRIME_data.csv', low_memory = False)
imp_df = pd.DataFrame(house_data)
prime_df = pd.DataFrame(PRIME_data)
imp_df.head()

,BATHRM,HF_BATHRM,AC,ROOMS,BEDRM,SALEDATE,KITCHENS,FIREPLACES,LANDAREA,LONGITUDE,LATITUDE,QUADRANT,PRICE,ZIPCODE,WARD,PRIME
0,3,1,Y,9,5,2016.0,2.0,4,1680,-77.040678,38.914684,NW,2100000.0,20009.0,2,3.750
1,3,1,Y,8,5,2006.0,2.0,3,1680,-77.040629,38.914683,NW,1602000.0,20009.0,2,7.875
2,3,2,Y,10,5,2010.0,1.0,4,2196,-77.039715,38.914331,NW,1950000.0,20009.0,2,3.250
3,3,1,Y,8,4,2011.0,2.0,1,1627,-77.040129,38.915408,NW,1050000.0,20009.0,2,3.250
4,3,1,Y,7,3,2018.0,2.0,1,1424,-77.039903,38.915017,NW,1430000.0,20009.0,2,5.125


In [4]:
imp_df = imp_df.drop(columns=['LONGITUDE','LATITUDE'])
imp_df.dtypes

BATHRM          int64
HF_BATHRM       int64
AC             object
ROOMS           int64
BEDRM           int64
SALEDATE      float64
KITCHENS      float64
FIREPLACES      int64
LANDAREA        int64
QUADRANT       object
PRICE         float64
ZIPCODE       float64
WARD            int64
PRIME         float64
dtype: object

In [5]:
imp_cat = imp_df.dtypes[imp_df.dtypes == "object"].index.tolist()


In [6]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(imp_df[imp_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(imp_cat)
encode_df.head()

C:\Users\FrantheMan\anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,AC_0,AC_N,AC_Y,QUADRANT_NE,QUADRANT_NW,QUADRANT_SE,QUADRANT_SW
0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,0.0,0.0,1.0,0.0,1.0,0.0,0.0


In [7]:
encode_df = encode_df.drop(columns = 'AC_0')
# Merge one-hot encoded features and drop the originals
imp_df = imp_df.merge(encode_df,left_index=True, right_index=True)
imp_df = imp_df.drop(imp_cat,1)
imp_df.head()

C:\Users\FrantheMan\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  after removing the cwd from sys.path.


,BATHRM,HF_BATHRM,ROOMS,BEDRM,SALEDATE,KITCHENS,FIREPLACES,LANDAREA,PRICE,ZIPCODE,WARD,PRIME,AC_N,AC_Y,QUADRANT_NE,QUADRANT_NW,QUADRANT_SE,QUADRANT_SW
0,3,1,9,5,2016.0,2.0,4,1680,2100000.0,20009.0,2,3.750,0.0,1.0,0.0,1.0,0.0,0.0
1,3,1,8,5,2006.0,2.0,3,1680,1602000.0,20009.0,2,7.875,0.0,1.0,0.0,1.0,0.0,0.0
2,3,2,10,5,2010.0,1.0,4,2196,1950000.0,20009.0,2,3.250,0.0,1.0,0.0,1.0,0.0,0.0
3,3,1,8,4,2011.0,2.0,1,1627,1050000.0,20009.0,2,3.250,0.0,1.0,0.0,1.0,0.0,0.0
4,3,1,7,3,2018.0,2.0,1,1424,1430000.0,20009.0,2,5.125,0.0,1.0,0.0,1.0,0.0,0.0


In [29]:
bins = []
val = 0
for i in range(30):
    base = 25000
    adding = adding + base
    bins.append(adding)



NameError: name 'adding' is not defined

In [ ]:
print(bins)   


In [12]:
bins = [25000, 50000, 75000,100000, 125000]
labels = [1,2,3,4]
cut = pd.cut(imp_df['PRICE'], bins=bins, labels=labels)
cut_df = pd.DataFrame(cut)
cut_df['REAL_PRICE'] = imp_df.PRICE
cut_df.head(50)

,PRICE,REAL_PRICE
0,NaN,2100000.0
1,NaN,1602000.0
2,NaN,1950000.0
3,NaN,1050000.0
4,NaN,1430000.0
5,NaN,1325000.0
6,NaN,1240000.0
7,NaN,592250.0
8,1,33232.0
9,NaN,907400.0


In [13]:
# Dropping rows with a saledate less than 2000
new_df = imp_df[imp_df.SALEDATE >= 2000]
new_df

,BATHRM,HF_BATHRM,ROOMS,BEDRM,SALEDATE,KITCHENS,FIREPLACES,LANDAREA,PRICE,ZIPCODE,WARD,PRIME,AC_N,AC_Y,QUADRANT_NE,QUADRANT_NW,QUADRANT_SE,QUADRANT_SW
0,3,1,9,5,2016.0,2.0,4,1680,2100000.0,20009.0,2,3.750,0.0,1.0,0.0,1.0,0.0,0.0
1,3,1,8,5,2006.0,2.0,3,1680,1602000.0,20009.0,2,7.875,0.0,1.0,0.0,1.0,0.0,0.0
2,3,2,10,5,2010.0,1.0,4,2196,1950000.0,20009.0,2,3.250,0.0,1.0,0.0,1.0,0.0,0.0
3,3,1,8,4,2011.0,2.0,1,1627,1050000.0,20009.0,2,3.250,0.0,1.0,0.0,1.0,0.0,0.0
4,3,1,7,3,2018.0,2.0,1,1424,1430000.0,20009.0,2,5.125,0.0,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55023,2,0,10,4,2004.0,2.0,0,5837,95000.0,20032.0,8,4.750,1.0,0.0,0.0,0.0,0.0,1.0
55024,2,0,10,4,2016.0,2.0,0,5302,185000.0,20032.0,8,3.750,0.0,1.0,0.0,0.0,0.0,1.0
55025,2,0,10,5,2012.0,2.0,0,5348,100000.0,20032.0,8,3.250,1.0,0.0,0.0,0.0,0.0,1.0
55026,2,0,10,4,2017.0,2.0,0,3046,215000.0,20032.0,8,4.250,1.0,0.0,0.0,0.0,0.0,1.0


In [14]:
remove_n = 35000
drop_indices = np.random.choice(new_df.index, remove_n, replace=False)
new_new_df = new_df.drop(drop_indices)
new_new_df

,BATHRM,HF_BATHRM,ROOMS,BEDRM,SALEDATE,KITCHENS,FIREPLACES,LANDAREA,PRICE,ZIPCODE,WARD,PRIME,AC_N,AC_Y,QUADRANT_NE,QUADRANT_NW,QUADRANT_SE,QUADRANT_SW
0,3,1,9,5,2016.0,2.0,4,1680,2100000.0,20009.0,2,3.750000,0.0,1.0,0.0,1.0,0.0,0.0
8,3,1,14,5,2016.0,1.0,3,2090,33232.0,20009.0,2,3.750000,0.0,1.0,0.0,1.0,0.0,0.0
9,1,0,6,3,2006.0,1.0,0,2090,907400.0,20009.0,2,7.875000,0.0,1.0,0.0,1.0,0.0,0.0
11,2,1,8,3,2012.0,1.0,2,2090,1100000.0,20009.0,2,3.250000,0.0,1.0,0.0,1.0,0.0,0.0
17,3,1,9,4,2016.0,2.0,2,1853,1550000.0,20009.0,2,3.750000,0.0,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55015,1,0,5,2,2008.0,1.0,1,1733,106000.0,20032.0,8,4.928571,0.0,1.0,0.0,0.0,1.0,0.0
55017,2,1,7,3,2008.0,1.0,1,1734,140496.0,20032.0,8,4.928571,0.0,1.0,0.0,0.0,1.0,0.0
55020,1,1,7,3,2013.0,1.0,0,2359,230000.0,20032.0,8,3.250000,0.0,1.0,0.0,0.0,1.0,0.0
55024,2,0,10,4,2016.0,2.0,0,5302,185000.0,20032.0,8,3.750000,0.0,1.0,0.0,0.0,0.0,1.0


In [15]:
# Define the features 
X = new_new_df.copy()
X = X.drop("PRICE", axis = 1)
X.head()

,BATHRM,HF_BATHRM,ROOMS,BEDRM,SALEDATE,KITCHENS,FIREPLACES,LANDAREA,ZIPCODE,WARD,PRIME,AC_N,AC_Y,QUADRANT_NE,QUADRANT_NW,QUADRANT_SE,QUADRANT_SW
0,3,1,9,5,2016.0,2.0,4,1680,20009.0,2,3.750,0.0,1.0,0.0,1.0,0.0,0.0
8,3,1,14,5,2016.0,1.0,3,2090,20009.0,2,3.750,0.0,1.0,0.0,1.0,0.0,0.0
9,1,0,6,3,2006.0,1.0,0,2090,20009.0,2,7.875,0.0,1.0,0.0,1.0,0.0,0.0
11,2,1,8,3,2012.0,1.0,2,2090,20009.0,2,3.250,0.0,1.0,0.0,1.0,0.0,0.0
17,3,1,9,4,2016.0,2.0,2,1853,20009.0,2,3.750,0.0,1.0,0.0,1.0,0.0,0.0


In [16]:
# Define target set
y = new_new_df['PRICE'].ravel()
y[:5]

array([2100000.,   33232.,  907400., 1100000., 1550000.])

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [18]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [19]:
# Create a random forest classifier
rf_model = RandomForestClassifier(n_estimators = 100, random_state = 1)

In [20]:
#Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [21]:
y_pred = rf_model.predict(X_test_scaled)

In [22]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test,y_pred))

                   pre       rec       spe        f1       geo       iba       sup

     5522.0       0.00      0.00      1.00      0.00      0.00      0.00         0
    15000.0       0.00      0.00      1.00      0.00      0.00      0.00         1
    21000.0       0.00      0.00      1.00      0.00      0.00      0.00         1
    25300.0       0.00      0.00      1.00      0.00      0.00      0.00         1
    28470.0       0.00      0.00      1.00      0.00      0.00      0.00         0
    30000.0       0.00      0.00      1.00      0.00      0.00      0.00         3
    33232.0       0.00      0.00      1.00      0.00      0.00      0.00         1
    35280.0       0.00      0.00      1.00      0.00      0.00      0.00         1
    35500.0       0.00      0.00      1.00      0.00      0.00      0.00         0
    38675.0       0.00      0.00      1.00      0.00      0.00      0.00         0
    40000.0       0.00      0.00      1.00      0.00      0.00      0.00         0
   

C:\Users\FrantheMan\anaconda3\envs\mlenv\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\FrantheMan\anaconda3\envs\mlenv\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\FrantheMan\anaconda3\envs\mlenv\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Sensitivity is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [23]:
acc_score = accuracy_score(y_test, y_pred)
acc_score

0.012433392539964476

In [24]:
# Displaying results
print("Confusion Matrix")
#display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, y_pred))

Confusion Matrix
Accuracy Score : 0.012433392539964476
Classification Report
              precision    recall  f1-score   support

      5522.0       0.00      0.00      0.00         0
     15000.0       0.00      0.00      0.00         1
     21000.0       0.00      0.00      0.00         1
     25300.0       0.00      0.00      0.00         1
     28470.0       0.00      0.00      0.00         0
     30000.0       0.00      0.00      0.00         3
     33232.0       0.00      0.00      0.00         1
     35280.0       0.00      0.00      0.00         1
     35500.0       0.00      0.00      0.00         0
     38675.0       0.00      0.00      0.00         0
     40000.0       0.00      0.00      0.00         0
     41500.0       0.00      0.00      0.00         1
     42000.0       0.00      0.00      0.00         1
     42900.0       0.00      0.00      0.00         1
     43000.0       0.00      0.00      0.00         1
     43500.0       0.00      0.00      0.00         1
    

C:\Users\FrantheMan\anaconda3\envs\mlenv\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\FrantheMan\anaconda3\envs\mlenv\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\FrantheMan\anaconda3\envs\mlenv\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Franth

In [25]:
# Calc feature importancy 
importancies = rf_model.feature_importances_
importancies

array([0.05644413, 0.05783078, 0.11563055, 0.07310436, 0.12101429,
       0.02352714, 0.04077885, 0.26468775, 0.06890483, 0.04038935,
       0.08217093, 0.01464229, 0.01493754, 0.01035633, 0.0062809 ,
       0.00766278, 0.0016372 ])

In [26]:
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.26468774578085025, 'LANDAREA'),
 (0.12101429133939291, 'SALEDATE'),
 (0.11563054856460972, 'ROOMS'),
 (0.08217092897015009, 'PRIME'),
 (0.0731043623499182, 'BEDRM'),
 (0.06890483187021292, 'ZIPCODE'),
 (0.05783077912327724, 'HF_BATHRM'),
 (0.056444130582889517, 'BATHRM'),
 (0.040778848542044645, 'FIREPLACES'),
 (0.04038934743151745, 'WARD'),
 (0.02352714297332784, 'KITCHENS'),
 (0.014937542462983573, 'AC_Y'),
 (0.014642290642415975, 'AC_N'),
 (0.010356327820528803, 'QUADRANT_NE'),
 (0.007662782567420094, 'QUADRANT_SE'),
 (0.006280901619095927, 'QUADRANT_NW'),
 (0.0016371973593649097, 'QUADRANT_SW')]